In [1]:
using Pkg
Pkg.activate("libs/")
Pkg.add("CSV")
# Pkg.add("DataFrames")
using JLD2
using DICOM, CSV, DataFrames


first_dir = "/media/molloi-lab/2TB"
BAC_root_dir = joinpath(first_dir, "Clean_Dataset_full");

  Activating project at `~/Desktop/Project BAC/BAC project/libs`


   Resolving package versions...


  No Changes to `~/Desktop/Project BAC/BAC project/libs/Project.toml`
  No Changes to `~/Desktop/Project BAC/BAC project/libs/Manifest.toml`


# 1. Read through all images and count how many GE images
read key (0x0008, 0x0070) and look for "GE MEDICAL SYSTEMS"

In [16]:
SIDs = readdir(BAC_root_dir)
s = size(SIDs)[1]
GE_SIDs = Array{String, 2}(undef, s, 2)
indexes = falses(s)
Threads.@threads for i = 1 : s
    sid = SIDs[i] 
    curr_dir = joinpath(BAC_root_dir, sid)
    for f in readdir(curr_dir)
        f_name, f_ext = splitext(f)
        if f_ext == ".dcm"
            # setup path
            img_path = joinpath(curr_dir, f)
            # read dcm
            dcm_data = dcm_parse(img_path)
            mnftr = dcm_data[(0x0008, 0x0070)]
            mnftr_model = dcm_data[(0x0008, 0x1090)]
            if mnftr == "GE MEDICAL SYSTEMS"
                GE_SIDs[i, 1] = sid
                GE_SIDs[i, 2] = mnftr_model
                indexes[i] = true
            else
                GE_SIDs[i, 1] = ""
                GE_SIDs[i, 2] = ""
            end
            break
        end
    end
end

GE_SIDs = GE_SIDs[indexes .== true, :]

1144×2 Matrix{String}:
 "SID-101917"  "Senograph DS ADS_43.10.1"
 "SID-101963"  "Senograph DS ADS_43.10.1"
 "SID-102042"  "Senograph DS ADS_43.10.1"
 "SID-102225"  "Senograph DS ADS_43.10.1"
 "SID-102422"  "Senograph DS ADS_43.10.1"
 "SID-102423"  "Senograph DS ADS_43.10.1"
 "SID-102433"  "Senograph DS ADS_43.10.1"
 "SID-102651"  "Senograph DS ADS_43.10.1"
 "SID-103306"  "Senographe Essential VERSION ADS_54.11"
 "SID-103971"  "Senograph DS ADS_43.10.1"
 ⋮             
 "SID-137486"  "Senograph DS VERSION ADS_54.11"
 "SID-137509"  "Senographe Essential VERSION ADS_54.11"
 "SID-137600"  "Senographe Essential VERSION ADS_54.11"
 "SID-137612"  "Senographe Essential VERSION ADS_54.11"
 "SID-137764"  "Senograph DS VERSION ADS_54.11"
 "SID-137773"  "Senograph DS VERSION ADS_54.11"
 "SID-137858"  "Senographe Essential VERSION ADS_54.11"
 "SID-138012"  "Senograph DS VERSION ADS_54.11"
 "SID-138027"  "Senographe Essential VERSION ADS_54.11"

In [20]:
unique(GE_SIDs[:, 2])

3-element Vector{String}:
 "Senograph DS ADS_43.10.1"
 "Senographe Essential VERSION ADS_54.11"
 "Senograph DS VERSION ADS_54.11"

# 2. Save GE SIDs as ".jld2" and ".csv"

In [17]:
@save "GE_SIDs.jld2" GE_SIDs

In [19]:
# Convert the array to a DataFrame
df = DataFrame(SID = GE_SIDs[:, 1], model = GE_SIDs[:, 2])

# Save the DataFrame to a CSV file
CSV.write("GE_images_info.csv", df)

"GE_images_info.csv"